## read source data

In [1]:
# from datetime import datetime
import pandas as pd

updateTime = pd.read_csv('https://health-infobase.canada.ca/src/data/covidLive/covid19-updateTime.csv',header = None).at[0,0]
worldData = pd.read_csv('https://mapdashbd.s3.ca-central-1.amazonaws.com/download/InternationalCovid19Cases.csv')
caInfoSource = pd.read_csv("https://health-infobase.canada.ca/src/data/covidLive/covid19.csv")
caInfoSource.to_csv('caInfo.csv')
worldData.to_csv('worldData.csv')

## ca total and new 

In [2]:
import pandas as pd
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

caInfo = pd.read_csv('caInfo.csv')
caTotal = pd.DataFrame(caInfo.loc[caInfo["pruid"] == 1, ["date", "numtotal", "numtoday"]])
ca_total_and_new_fig = make_subplots(rows=2, cols=1, subplot_titles=("Total confirmed", "New confirmed"))
ca_total_and_new_fig.add_trace(
   go.Scatter(
       x = pd.to_datetime(caTotal["date"], format="%d-%m-%Y"), 
       y = caTotal["numtotal"],
       mode='lines',
       name='',
       hovertemplate ='%{x}<br>' + '%{y:.2s}',
       showlegend = False,
   ),
    row=1,
    col=1,
)
ca_total_and_new_fig.add_trace(
   go.Bar(
       x = pd.to_datetime(caTotal["date"], format="%d-%m-%Y"), 
       y = caTotal["numtoday"],
       hovertemplate ='%{x}<br>' + '%{y:.2s}',
       name='',
       showlegend = False,
   ),
    row=2,
    col=1,
)
ca_total_and_new_fig.update_layout(height=550, width=600,template="plotly_white",)
#ca_total_and_new_fig.show()
#plotly.offline.plot(ca_total_and_new_fig, filename='./charts/ca_total_and_new_line.html', auto_open=False)
with open('1.html', 'a') as f:
    f.write(ca_total_and_new_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    #f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    #f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))


## world total line chart

In [3]:
import pandas as pd
import plotly.graph_objects as go

def get_date_world_total_line():
    worldInfo_df = pd.read_csv('worldData.csv')
    groupByCountry = worldInfo_df.groupby(worldInfo_df['id'])
    country_count = 0
    worldFig = go.Figure()
    for country, group in groupByCountry:
        country_count += 1
        tempDf = pd.DataFrame(data = group)
        tempDf['cumsum_cases'] = tempDf['cases'].cumsum()
        tempDf['cumsum_deaths'] = tempDf['deaths'].cumsum()
        worldFig.add_trace(
            go.Scatter(
                x=group['date'], 
                y=group['cumsum_cases'], 
                mode='lines', 
                name = country,
                hovertemplate =group['name_en']+'<br>%{y:.2s}'+'<br>%{x}',
            )
        )
        worldFig.add_trace(
            go.Scatter(
                x=group['date'], 
                y=group['cumsum_deaths'], 
                mode='lines', 
                name = country,
                hovertemplate =group['name_en']+'<br>%{y:.2s}'+'<br>%{x}',
                visible=False,
            )
        )
    worldFig.update_layout(
        title=dict(text='Total cases of covid-19 by country',font=dict(size=16),x=0.5,y=0.95,yanchor='top', xanchor='center'),
        font={"size":12},
        width=600,
        height=550,
        margin=dict(t=100, b=20, l=0, r=0),
        yaxis_title='Total amount',
        xaxis_title='Time',
        template="plotly_white",
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                x=0.7,
                y=1.1,
                buttons=list([
                    dict(label="total cases",
                         method="update",
                         args=[{"visible": [True, False,] * ((int)(country_count / 2))},
                               {"title": "Total cases of covid-19 by country",
                                "annotations": []}]),
                    dict(label="total deaths",
                         method="update",
                         args=[{"visible": [False, True] * ((int)(country_count / 2))},
                               {"title": "Total deaths of covid-19 by country",
                                "annotations": []}]),
                ]),
            )
        ]
    )
    #worldFig.show()
    plotly.offline.plot(worldFig, filename='./charts/world_line.html', auto_open=False)

    
get_date_world_total_line()


## world data-date bar chart with slidebar and animation

In [4]:
import pandas as pd
import plotly.express as px    
#set cumsum of each country
worldInfo_df = pd.read_csv('worldData.csv')
worldDataUpdate = worldInfo_df[['date', 'name_en', 'deaths', 'cases']].copy()
worldDataUpdate['cumsum'] = worldDataUpdate.groupby(worldDataUpdate['name_en'])['cases'].cumsum()
worldDataUpdate['date'] = pd.to_datetime(worldDataUpdate['date'], format = '%Y-%m-%d')
#get unique countries and color list
countries = worldDataUpdate['name_en'].unique()
countries_count = len(countries)
colors1 = px.colors.qualitative.Pastel
colors2 = px.colors.qualitative.Vivid
colors3 = px.colors.qualitative.Dark2
color_list = (colors1 + colors2 + colors3)*9
color_list = color_list[:countries_count]

#set color for each country
for i in range(countries_count):
        worldDataUpdate.loc[worldDataUpdate['name_en'] == countries[i], 'color'] = color_list[i] 
        
#save update into csv
worldDataUpdate.to_csv("worldData_update.csv")

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import datetime

date_country_cumsum_dict_slider = {} #key: date, value: dataframe of date,name,cumsum

def get_world_data_slider():
    global date_country_cumsum_dict_slider
    worldInfo_df = pd.read_csv('worldData_update.csv')
    worldDataUpdate = worldInfo_df[['date', 'name_en', 'deaths', 'cases','cumsum','color']].copy()
    dates = np.sort(pd.unique(worldDataUpdate['date']))
    for date in dates:
        countries_cumsum_df = worldDataUpdate.loc[worldDataUpdate['date'] == date]
        countries_cumsum_df = countries_cumsum_df.nlargest(20, 'cumsum').sort_values(by = ['cumsum'], ascending = True)
        date_country_cumsum_dict_slider[date] = countries_cumsum_df

def get_world_fig_slider():
    get_world_data_slider()
    # Add traces, one for each slider step
    count = 0
    worldFig = go.Figure()
    for date in date_country_cumsum_dict_slider:
        count += 1
        tempDf = date_country_cumsum_dict_slider[date]
        worldFig.add_trace(
            go.Bar(
                name = "",
                text =tempDf['cumsum'],
                textfont = dict(size = 14, family = 'Droid Sans Mono'),
                textposition = "inside",
                texttemplate = '%{text:.3s}',
                visible = False,
                marker = dict(color = tempDf['color']),
                x = tempDf['cumsum'],
                y = tempDf['name_en'],
                orientation='h',
            )
        )
    worldFig.data[count-1].visible = True

    # Create and add slider
    figSize = len(worldFig.data)
    steps = []
    visibleIndex = 0
    for date in date_country_cumsum_dict_slider:
        tempDf = date_country_cumsum_dict_slider[date]
        step = dict(
        method = 'update',
        args = [{"visible": [False] * figSize}], #set all traces to invisible
        label = str(pd.to_datetime(date).date())
        )
        step['args'][0]["visible"][visibleIndex] = True #set one day's trace to visible. visibleIndex0 <=> date0, visibleIndex1 <=> date1 ...
        visibleIndex += 1
        steps.append(step)
    sliders = [dict(
        active = visibleIndex-1,
        steps = steps,
        pad=dict(t=15,b=20,l=0,r=25)
    )]

    #print(steps[visibleIndex - 2]["args"])
    worldFig.update_layout(
        sliders = sliders,
        title=dict(text='Time line chart of total cases',font=dict(size=16),x=0.5,y=0.95,yanchor='top', xanchor='center'),
        title_x=0.5,
        font={"size":12},
        width=600,
        height=550,
        margin=dict(t=60, b=0, l=0, r=0),
        template="plotly_white",
    )
    #worldFig.show()      
    plotly.offline.plot(worldFig, filename='./charts/world_total_slider.html', auto_open=False)


get_world_fig_slider()

In [6]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

date_country_cumsum_dict_animation = {} #key: date, value: dataframe of date,name,cumsum
def get_world_data_animation():
    global date_country_cumsum_dict_animation
    worldInfo_df = pd.read_csv('worldData_update.csv')
    worldDataUpdate = worldInfo_df[['date', 'name_en', 'deaths', 'cases','cumsum','color']].copy()
    dates = np.sort(pd.unique(worldDataUpdate['date']))
    for date in dates:
        countries_cumsum_df = worldDataUpdate.loc[worldDataUpdate['date'] == date]
        countries_cumsum_df = countries_cumsum_df.nlargest(20, 'cumsum').sort_values(by = ['cumsum'], ascending = True)
        date_country_cumsum_dict_animation[date] = countries_cumsum_df
        
def get_world_fig_animation():
    get_world_data_animation()
    frames = []
    for date in date_country_cumsum_dict_animation:
        tempDf = date_country_cumsum_dict_animation[date]
        maxCumsum = float(tempDf['cumsum'].max()) * 1.3 
        dateString = str(pd.to_datetime(date).date())
        frame = go.Frame(
            data = [go.Bar(
                name = "",
                text =tempDf['cumsum'],
                textfont = dict(size = 14, family = 'Droid Sans Mono'),
                textposition = "inside",
                texttemplate = '%{text:.3s}',
                marker = dict(color = tempDf['color']),
                x = date_country_cumsum_dict_animation[date]['cumsum'],
                y = date_country_cumsum_dict_animation[date]['name_en'],
                orientation='h',
            )],
             layout = go.Layout(
                 xaxis = dict(range=[0,maxCumsum], autorange=False),
                 title = dict(text=('Animation of total confirmed numbers per country: ' + dateString), font=dict(size=18),x=0.5,xanchor='center'),
             )
        )
        frames.append(frame)

    worldFig = go.Figure( 
        data=[go.Bar(
                name = "",
                marker = dict(color = tempDf['color']),
                x = date_country_cumsum_dict_animation['2019-12-31']['cumsum'],
                y = date_country_cumsum_dict_animation['2019-12-31']['name_en'],
                orientation='h',
            )],
        layout=go.Layout(
            title=dict(text='Animation of total confirmed numbers per country: 2019-12-31',font=dict(size=16),x=0.5,y=0.95,yanchor='top',xanchor='center'),
            font={"size":12},
            width=600,
            height=550,
            xaxis=dict(range=[0,40000], autorange=False),
            margin=dict(t=100, b=20, l=0, r=0),
            template="plotly_white",     
            updatemenus=[dict(
                type="buttons",
                direction="left",
                x=0.5,
                y=1.1,
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None,
                              {"frame": {"duration": 500, "redraw": True},
                                "fromcurrent": True,
                              "transition": {"duration":500,
                              "easing": "linear"}}]
                ),
                        dict(label="Pause",
                              method="animate",
                              args=[[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}]
                )]
            )],
         ),
        frames = frames,
    )
    #worldFig.show()
    plotly.offline.plot(worldFig, filename='./charts/world_total_animation.html', auto_open=False)

get_world_fig_animation()

## world data pie chart

In [7]:
##### import pandas as pd
import plotly
import plotly.graph_objects as go

worldData_pie_cumsum_and_death_df_update = pd.DataFrame() #top 20 with sum of rest countries
worldData_pie_cumsum_and_death_df_update_rest = pd.DataFrame() #rest countries
pie_day = None

def get_pie_update():
    worldInfo_df = pd.read_csv('worldData.csv')
    worldData_pie_cumsum_and_death_df = worldInfo_df[['date', 'name_en', 'deaths', 'cases']].copy() #copy data
    global pie_day 
    pie_day = worldData_pie_cumsum_and_death_df['date'].max() #get latest date
    worldData_pie_cumsum_and_death_df['cumsum'] = worldData_pie_cumsum_and_death_df.groupby('name_en')['cases'].cumsum() #calculate cumsum
    worldData_pie_cumsum_and_death_df = worldData_pie_cumsum_and_death_df.loc[worldData_pie_cumsum_and_death_df['date'] == pie_day][['name_en', 'cumsum', 'deaths']] #get latest date's data
    worldData_pie_cumsum_and_death_df['rank'] = worldData_pie_cumsum_and_death_df['cumsum'].rank(ascending = False) #calculate cumsum's rank
    worldData_pie_cumsum_and_death_df.columns = ['country', 'total_confirmed', 'deaths', 'rank'] #rename columns
    #calculate not-top-20 countries total confirmed and deaths
    restTotal = 0
    restDeath = 0
    for index, row in worldData_pie_cumsum_and_death_df.iterrows():
        if row['rank'] > 20.0:
            restTotal += row['total_confirmed']
            restDeath += row['deaths']
    global worldData_pie_cumsum_and_death_df_update, worldData_pie_cumsum_and_death_df_update_rest
    #create new df that does not include countries that are not top 20
    worldData_pie_cumsum_and_death_df_update = worldData_pie_cumsum_and_death_df.loc[worldData_pie_cumsum_and_death_df['rank'] <= 20]
    #append other countries data
    temp = pd.DataFrame([['other', restTotal, restDeath, 21.0]], columns = ['country', 'total_confirmed', 'deaths', 'rank'])
    worldData_pie_cumsum_and_death_df_update = worldData_pie_cumsum_and_death_df_update.append(temp, ignore_index=True)
    #create new df that only include countries that are not top 20
    worldData_pie_cumsum_and_death_df_update_rest = worldData_pie_cumsum_and_death_df.loc[worldData_pie_cumsum_and_death_df['rank'] > 20]
    
def get_world_fig_pie():
    get_pie_update()
    #create pie chart of total confirmed and deaths
    world_pie_fig = go.Figure(
        layout=go.Layout(
            title=dict(text='Total confirmed numbers as of ' + pie_day,font=dict(size=16),x=0.35,y=0.95,yanchor='top',xanchor='center')
    ))
    world_pie_fig.add_trace(
        go.Pie(
            #title=dict(text='Total confirmed numbers as of ' + pie_day + '<br>World', font=dict(size=16)),
            labels = worldData_pie_cumsum_and_death_df_update['country'],
            values = worldData_pie_cumsum_and_death_df_update['total_confirmed'],
            textinfo='label+percent',
            textposition='inside',
            hoverinfo='label+value+percent',
        )
    )

    world_pie_fig.add_trace(
        go.Pie(
            #title=dict(text='Total confirmed numbers as of ' + pie_day + '<br>Other', font=dict(size=16)),
            labels = worldData_pie_cumsum_and_death_df_update_rest['country'],
            values = worldData_pie_cumsum_and_death_df_update_rest['total_confirmed'],
            textinfo='label+percent',
            textposition='inside',
            visible=False,
            hoverinfo='label+value+percent',
        )
    )
    world_pie_fig.update_layout(
        font={"size":12},
        width=600,
        height=550,
        margin=dict(t=60, b=0, l=0, r=0),
        template="plotly_white", 
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                active=0,
                x=0.65,
                y=1,
                buttons=list([
                    dict(label="world",
                         method="update",
                         args=[{"visible": [True, False,]},]),
                    dict(label="other",
                         method="update",
                         args=[{"visible": [False, True]},]),
                ]),
            )
        ])
    #world_pie_fig.show()
    plotly.offline.plot(world_pie_fig, filename='./charts/world_pie.html', auto_open=False)
    
get_world_fig_pie()


In [8]:
a = [1,2,3,4]
i=1
print(a[--i])
print(i)

2
1
